In [1]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

# Load environment variables from .env file
load_dotenv()

# Read Neo4j credentials from environment variables
neo4j_username = os.getenv('NEO4J_USERNAME')
neo4j_password = os.getenv('NEO4J_PASSWORD')
neo4j_uri = os.getenv('NEO4J_URI')

# Create a Neo4j driver instance
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_username, neo4j_password))

# Verify the connection

def verify_connection(driver):
    try:
        with driver.session() as session:
            result = session.run("RETURN 1")
            if result.single()[0] == 1:
                print("Connection to Neo4j established successfully.")
            else:
                print("Failed to establish connection to Neo4j.")
    except Exception as e:
        print(f"An error occurred: {e}")

verify_connection(driver)



Connection to Neo4j established successfully.


In [2]:
def get_schema(driver):
    schema = {"nodes": [], "edges": []}
    try:
        with driver.session() as session:
            # Get node labels
            node_labels_result = session.run("CALL db.labels()")
            schema["nodes"] = [record["label"] for record in node_labels_result]
            
            # Get relationship types
            relationship_types_result = session.run("CALL db.relationshipTypes()")
            schema["edges"] = [record["relationshipType"] for record in relationship_types_result]
    except Exception as e:
        print(f"An error occurred while retrieving the schema: {e}")
    return schema

schema = get_schema(driver)
print(schema)


{'nodes': ['Case', 'Party', 'Justice', 'Advocate', 'Opinion', 'ORG', 'LOC', 'PER', 'MISC'], 'edges': ['case_opinion', 'alternate_name', 'mentioned_in', 'Petitioner', 'Respondent', 'advocated_by', 'decided_by', 'won_by', 'majority', 'minority', 'based_in', 'none', 'lived_in', 'religion', 'employee_of', 'works_for', 'parent_of', 'child_of', 'has_shareholder', 'owns', 'sibling_of', 'related_to', 'Appellant', 'Appellee', 'charges', 'origin', 'born_in', 'school_attended', 'has_title', 'part_of', 'website']}


In [ ]:
def get_most_connected_nodes_and_edges(driver, limit=10):
    query = """
    MATCH (n)-[r]->()
    RETURN id(n) as node_id, labels(n) as node_labels, COUNT(r) as connections
    ORDER BY connections DESC
    LIMIT $limit
    """
    nodes = []
    edges = []
    try:
        with driver.session() as session:
            result = session.run(query, limit=limit)
            for record in result:
                node_id = record["node_id"]
                node_labels = record["node_labels"]
                connections = record["connections"]
                nodes.append({"node_id": node_id, "node_labels": node_labels})
                
                # Get all edges for the current node
                edge_query = """
                MATCH (n)-[r]->(m)
                WHERE id(n) = $node_id
                RETURN id(r) as edge_id, id(m) as connected_node_id
                """
                edge_result = session.run(edge_query, node_id=node_id)
                for edge_record in edge_result:
                    edge_id = edge_record["edge_id"]
                    connected_node_id = edge_record["connected_node_id"]
                    edges.append({"edge_id": edge_id, "connected_node_id": connected_node_id})
    except Exception as e:
        print(f"An error occurred while retrieving the most connected nodes and edges: {e}")
    return {"nodes": nodes, "edges": edges}

most_connected_nodes_and_edges = get_most_connected_nodes_and_edges(driver)
print(most_connected_nodes_and_edges)
